<a href="https://colab.research.google.com/github/Sakinat-Folorunso/CMP_805_Advanced_Programming_Languages/blob/main/notebooks/CMP805_Week6_PH_Python_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMP805 — Week 6 Practical (Python, Colab)
**Topic:** Functional programming — evaluation order (strict vs lazy), combinators & folds, property‑based tests  
**Course:** Advanced Programming Languages (M.Sc.), OOU — CMP805

**Instructor:** **DR SAKINAT FOLORUNSO – ASSOCIATE PROFESSOR OF AI SYSTEMS AND FAIR DATA**  
**Department:** **COMPUTER SCIENCES, OLABISI ONABANJO UNIVERSITY, AGO‑IWOYE, OGUN STATE, NIGERIA**

> This PH mirrors your outline’s Week‑6 plan by practicing **combinators & folds** and using **property‑based testing** to validate laws. We also simulate **laziness** with generators/streams in Python.

### Learning goals (≈60 minutes)
- Implement higher‑order **combinators**: `compose`, `map_fn`, `filter_fn`, `foldl`, `foldr`.
- Simulate **lazy evaluation** via Python generators and a tiny `Stream` wrapper.
- Use **Hypothesis** to check properties (map fusion, filter distributivity, fold/sum agreement).

In [ ]:
# 🧑‍🎓 Student info
STUDENT_NAME = "Type your full name here"
STUDENT_ID   = "Matric/ID here"
print("Student:", STUDENT_NAME, "| ID:", STUDENT_ID)

In [ ]:
# ✅ Environment check
import sys
major, minor = sys.version_info[:2]
assert (major, minor) >= (3, 10), f"Need Python 3.10+, found {major}.{minor}"
print(f"Python {major}.{minor} OK — match/case available.")

In [ ]:
# 📦 Install property-based testing library
try:
    import hypothesis, hypothesis.strategies as st  # noqa: F401
except Exception:
    %pip -q install hypothesis
    import hypothesis, hypothesis.strategies as st  # noqa: F401
print("Hypothesis available.")

In [ ]:
# =====================================
# 🧱 Part 1 — Combinators & folds
# =====================================
from typing import Callable, Iterable, TypeVar, List, Iterator, Any

A = TypeVar("A")
B = TypeVar("B")
C = TypeVar("C")

def compose(f: Callable[[B], C], g: Callable[[A], B]) -> Callable[[A], C]:
    """Function composition: (f ∘ g)(x) = f(g(x))"""
    def h(x: A) -> C:
        return f(g(x))  # apply g first, then f
    return h

def map_fn(f: Callable[[A], B], xs: Iterable[A]) -> List[B]:
    """Pure map producing a list (deterministic over the given iterable)."""
    out: List[B] = []
    for x in xs:
        out.append(f(x))
    return out

def filter_fn(p: Callable[[A], bool], xs: Iterable[A]) -> List[A]:
    """Pure filter producing a list of elements where predicate p holds."""
    out: List[A] = []
    for x in xs:
        if p(x):
            out.append(x)
    return out

def foldl(f: Callable[[B, A], B], z: B, xs: Iterable[A]) -> B:
    """Left fold: (((z ⊕ x1) ⊕ x2) ⊕ ...). Strict in Python by nature."""
    acc: B = z
    for x in xs:
        acc = f(acc, x)
    return acc

def foldr(f: Callable[[A, B], B], z: B, xs: List[A]) -> B:
    """Right fold over a finite list: x1 ⊕ (x2 ⊕ (... ⊕ (xn ⊕ z)))."""
    acc: B = z
    for x in reversed(xs):
        acc = f(x, acc)
    return acc

# Sanity checks (deterministic laws on small data)
assert map_fn(lambda x: x+1, [1,2,3]) == [2,3,4]
assert filter_fn(lambda x: x%2==0, [1,2,3,4]) == [2,4]
assert foldl(lambda a,b: a+b, 0, [1,2,3]) == 6
assert foldr(lambda x,a: x+a, 0, [1,2,3]) == 6
print("ok  - basic combinator sanity checks")

In [ ]:
# =====================================
# 💤 Part 2 — Laziness via generators
# =====================================
from itertools import islice
from typing import Optional

def naturals(start: int = 0) -> Iterator[int]:
    """Infinite generator of natural numbers: start, start+1, ..."""
    n = start
    while True:
        yield n
        n += 1

def take(n: int, it: Iterable[A]) -> List[A]:
    """Take first n items from an iterable (lazy-friendly)."""
    return list(islice(it, n))

# Example: sum of first 5 squares without creating an infinite list in memory
def squares(it: Iterable[int]) -> Iterator[int]:
    for x in it:
        yield x*x

first5 = take(5, squares(naturals(1)))
print("first5 squares:", first5, "| sum =", sum(first5))

# Minimal Stream wrapper that delays tails
class Stream:
    def __init__(self, head: A, tail_thunk: Optional[Callable[[], "Stream"]]):
        self.head = head
        self._tail_thunk = tail_thunk
        self._tail = None

    @property
    def tail(self) -> "Stream | None":
        if self._tail_thunk is None:
            return None
        if self._tail is None:
            self._tail = self._tail_thunk()  # compute on first demand
        return self._tail

def stream_from(n: int) -> Stream:
    return Stream(n, lambda: stream_from(n+1))

def stream_map(f: Callable[[A], B], s: "Stream | None") -> "Stream | None":
    if s is None: return None
    return Stream(f(s.head), lambda: stream_map(f, s.tail))

def stream_take(n: int, s: "Stream | None") -> List[A]:
    out: List[A] = []
    cur = s
    for _ in range(n):
        if cur is None: break
        out.append(cur.head)
        cur = cur.tail
    return out

print("stream first 5:", stream_take(5, stream_from(1)))
print("stream squares first 5:", stream_take(5, stream_map(lambda x: x*x, stream_from(1))))

In [ ]:
# =====================================
# 🧪 Part 3 — Property-based testing
# =====================================
from hypothesis import given
import hypothesis.strategies as st

# Strategy for integer lists
ints = st.lists(st.integers(min_value=-100, max_value=100), max_size=30)

@given(ints)
def test_foldl_sum(xs):
    assert foldl(lambda a,b: a+b, 0, xs) == sum(xs)

@given(ints)
def test_foldr_sum(xs):
    assert foldr(lambda x,a: x+a, 0, xs) == sum(xs)

# Map fusion: map f (map g xs) == map (f∘g) xs  (test a small function set)
funcs = st.sampled_from([lambda x: x+1, lambda x: x*2, lambda x: -x])

@given(funcs, funcs, ints)
def test_map_fusion(f, g, xs):
    left  = map_fn(f, map_fn(g, xs))
    right = map_fn(compose(f,g), xs)
    assert left == right

# Filter distributivity: filter p (filter q xs) == filter (lambda x: p(x) and q(x)) xs
preds = st.sampled_from([lambda x: x%2==0, lambda x: x>0, lambda x: x%3!=0])

@given(preds, preds, ints)
def test_filter_and(p, q, xs):
    left  = filter_fn(p, filter_fn(q, xs))
    right = filter_fn(lambda x: p(x) and q(x), xs)
    assert left == right

print("Running property tests…")
test_foldl_sum(); test_foldr_sum(); test_map_fusion(); test_filter_and()
print("ok  - property tests passed")

### 🧪 Your Turn (10–15 minutes)
1. Implement `Sub(a, b)` and adapt your property tests: show that `foldl` with subtraction is **not** generally equal to `foldr` on the same list.  
2. Write a generator `fib()` yielding the Fibonacci sequence and demonstrate laziness by printing only the first 10 values without computing beyond that.

### ✍️ Reflection (2–3 sentences)
- When is **laziness** preferable to **strict** evaluation? Give one example from this lab.  
- Which property was most surprising to you and why?

In [ ]:
# 📝 Save small submission bundle
import json, time
stamp = time.strftime("%Y-%m-%d %H:%M:%S")
submission = {
    "student_name": STUDENT_NAME,
    "student_id": STUDENT_ID,
    "timestamp": stamp,
    "properties": ["foldl==sum", "foldr==sum", "map fusion", "filter ∧ distributivity"],
    "reflection": "(fill in here)"
}
with open("week6_submission.json", "w") as f:
    json.dump(submission, f, indent=2)
print("Saved week6_submission.json — upload with your notebook.")